In [1]:
import torchvision
import torch
import torch.nn.utils.prune as prune
import copy

original_model = torchvision.models.mobilenet_v2(pretrained=True)
#
# Saves the original model and works on a pruned copy
#
pruned_model = copy.deepcopy(original_model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 68.3MB/s]


**Evaluate function**

In [2]:
import time
import numpy as np
def evaluate(model, data_loader, loss_history):
    model.eval()

    total_samples = len(data_loader.dataset)
    correct_samples = 0
    total_loss = 0
    times = []
    with torch.no_grad():
        for data, target in data_loader:
            start_time = time.time()
            output = torch.nn.functional.log_softmax(model(data), dim=1)
            end_time = time.time()
            times.append(1000*(end_time - start_time))
            loss = torch.nn.functional.nll_loss(output, target, reduction='sum')
            _, pred = torch.max(output, dim=1)
            total_loss += loss.item()
            correct_samples += pred.eq(target).sum()
    avg_inference = np.mean(times)
    std_dev_inference = np.std(times)
    min_inference = np.min(times)
    max_inference = np.max(times)

    avg_loss = total_loss / total_samples
    loss_history.append(avg_loss)
    print('\tAverage test loss: ' + '{:.4f}'.format(avg_loss) +
          '\tAccuracy:' + '{:5}'.format(correct_samples) + '/' +
          '{:5}'.format(total_samples) + ' (' +
          '{:4.2f}'.format(100.0 * correct_samples / total_samples) + '%)' +
          '\tAverage inference time: ' + '{:.4f}ms'.format(avg_inference) +'\n')

**Evaluate function that will provde the assesment between a model and a dataset.**

In [3]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from google.colab import drive
from torch.utils.data import Subset
drive.mount('/content/drive')

data_path = "/content/drive/My Drive/colab_files/imagenet/"
imagenet_val = datasets.ImageNet(
	root=data_path,
	split='val',
    transform=transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225])
	])
)

Mounted at /content/drive


**Create Data Loader**

In [4]:
from random import randint
maxSample = 150
indexes = [randint(1, len(imagenet_val)) for _ in range(maxSample)]
dataset_subset = Subset(imagenet_val,indexes)

data_loader_subset = torch.utils.data.DataLoader(dataset_subset,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=2)

In [5]:
for module in pruned_model.modules():
  print("Module: " + str(module))

Module: MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(

**Unstructured Pruning Using L1 Norm with a ratio of 5%**

In [7]:
for name, module in pruned_model.named_modules():
    # prune 05% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
      sparsity1 = float(torch.sum(module.weight == 0)) / float(module.weight.nelement())
      #print("Previous sparsity: {:.2f}%".format(sparsity1))
      #print(module.weight)
      prune.l1_unstructured(module, name='weight', amount=0.05)
      prune.remove(module,'weight')
      sparsity2 = float(torch.sum(module.weight == 0)) / float(module.weight.nelement())
      #print("Module name: " + name)
      print("Sparsity after pruning: {:.2f}%".format(sparsity2))
      #print(module.weight)
      #input()
loss=[]
print("Non-Pruned Model Metrics:\n")
evaluate(original_model,data_loader_subset,loss)
loss=[]
print("Pruned Model Metrics:\n")
evaluate(pruned_model,data_loader_subset,loss)
pruned_model = copy.deepcopy(original_model)





Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity after pruning: 0.05%
Sparsity a

**Unstructured Pruning Using L1 Norm with a ratio of 10%**

In [8]:
for name, module in pruned_model.named_modules():
    # prune 10% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
      prune.l1_unstructured(module, name='weight', amount=0.1)
      prune.remove(module,'weight')

loss=[]
print("Non-Pruned Model Metrics:\n")
evaluate(original_model,data_loader_subset,loss)
loss=[]
print("Pruned Model Metrics:\n")
evaluate(pruned_model,data_loader_subset,loss)
pruned_model = copy.deepcopy(original_model)

Non-Pruned Model Metrics:

	Average test loss: 1.0436	Accuracy:  111/  150 (74.00%)	Average inference time: 230.3502ms

Pruned Model Metrics:

	Average test loss: 1.1681	Accuracy:  105/  150 (70.00%)	Average inference time: 200.7136ms



**Unstructured Pruning Using L1 Norm with a ration of 15%**

In [9]:
for name, module in pruned_model.named_modules():
    # prune 15% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
      prune.l1_unstructured(module, name='weight', amount=0.15)
      prune.remove(module,'weight')

#loss=[]
#print("Non-Pruned Model Metrics:\n")
#evaluate(original_model,data_loader_subset,loss)
loss=[]
print("Pruned Model Metrics:\n")
evaluate(pruned_model,data_loader_subset,loss)
pruned_model = copy.deepcopy(original_model)

Pruned Model Metrics:

	Average test loss: 1.4010	Accuracy:   98/  150 (65.33%)	Average inference time: 187.0141ms



**Unstructured Pruning Using L1 Norm with a ratio of 20%**

In [10]:
for name, module in pruned_model.named_modules():
    # prune 20% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
      prune.l1_unstructured(module, name='weight', amount=0.20)
      prune.remove(module,'weight')
#loss=[]
#print("Non-Pruned Model Metrics:\n")
#evaluate(original_model,data_loader_subset,loss)
loss=[]
print("Pruned Model Metrics:\n")
evaluate(pruned_model,data_loader_subset,loss)
pruned_model = copy.deepcopy(original_model)

Pruned Model Metrics:

	Average test loss: 1.8865	Accuracy:   84/  150 (56.00%)	Average inference time: 186.1215ms



**Unstructured Pruning Using Random with a ratio of 5%**

In [11]:
for name, module in pruned_model.named_modules():
    # prune 05% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
      prune.random_unstructured(module, name='weight', amount=0.05)
      prune.remove(module,'weight')
#loss=[]
#print("Non-Pruned Model Metrics:\n")
#evaluate(original_model,data_loader_subset,loss)
loss=[]
print("Pruned Model Metrics:\n")
evaluate(pruned_model,data_loader_subset,loss)
pruned_model = copy.deepcopy(original_model)

Pruned Model Metrics:

	Average test loss: 8.3882	Accuracy:    0/  150 (0.00%)	Average inference time: 179.5899ms



**Unstructured Pruning Using Random with a ratio of 10%**

In [12]:
for name, module in pruned_model.named_modules():
    # prune 10% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
      prune.random_unstructured(module, name='weight', amount=0.10)
      prune.remove(module,'weight')
#loss=[]
#print("Non-Pruned Model Metrics:\n")
#evaluate(original_model,data_loader_subset,loss)
loss=[]
print("Pruned Model Metrics:\n")
evaluate(pruned_model,data_loader_subset,loss)
pruned_model = copy.deepcopy(original_model)

Pruned Model Metrics:

	Average test loss: 13.0088	Accuracy:    1/  150 (0.67%)	Average inference time: 211.2055ms



**Structured Pruning Using L1 Norm with a ratio of 5% across rows**

In [13]:
for name, module in pruned_model.named_modules():
    # prune 20% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
      #
      # n is the order of the norm.  In this case we are using L1 norm (n=1)
      # dim is the index of the dimension along which we define channels to prune.
      # - dimension=0: This means you are pruning along the rows of the weight tensor.
      #   In this case, weights are assessed across different input features, and
      #    entire rows may be removed based on their norm.
      # - dimension=1: This means you are pruning along the columns. Here, weights
      #    are evaluated across different output features, allowing for entire columns to be pruned.
      prune.ln_structured(module, name="weight", amount=0.05, n=1, dim=0)
      prune.remove(module,'weight')
#loss=[]
#print("Non-Pruned Model Metrics:\n")
#evaluate(original_model,data_loader_subset,loss)
loss=[]
print("Pruned Model Metrics:\n")
evaluate(pruned_model,data_loader_subset,loss)
pruned_model = copy.deepcopy(original_model)

Pruned Model Metrics:

	Average test loss: 8.8327	Accuracy:    0/  150 (0.00%)	Average inference time: 213.7322ms



**Structured Pruning Using L1 Norm with a ratio of 5% across columns**

In [14]:
for name, module in pruned_model.named_modules():
    # prune 20% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
      #
      # n is the order of the norm.  In this case we are using L1 norm (n=1)
      # dim is the index of the dimension along which we define channels to prune.
      # - dimension=0: This means you are pruning along the rows of the weight tensor.
      #   In this case, weights are assessed across different input features, and
      #    entire rows may be removed based on their norm.
      # - dimension=1: This means you are pruning along the columns. Here, weights
      #    are evaluated across different output features, allowing for entire columns to be pruned.
      prune.ln_structured(module, name="weight", amount=0.05, n=1, dim=1)
      prune.remove(module,'weight')
#loss=[]
#print("Non-Pruned Model Metrics:\n")
#evaluate(original_model,data_loader_subset,loss)
loss=[]
print("Pruned Model Metrics:\n")
evaluate(pruned_model,data_loader_subset,loss)
pruned_model = copy.deepcopy(original_model)

Pruned Model Metrics:

	Average test loss: 2.9679	Accuracy:   60/  150 (40.00%)	Average inference time: 194.1096ms

